In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd

import align
import argparse
import codecs
import os, sys
from random import random, choice
import re
import glob
import matplotlib.pyplot as plt
import time


from IPython.utils import io

In [2]:
max_stem_length = 10
def read_data(filename):
    with codecs.open(filename, 'r', 'utf-8') as inp:
        lines = inp.readlines()
    inputs = []
    outputs = []
    tags = []
    for l in lines:
        l = l.strip().split('\t')
        if l:
            inputs.append(list(l[0].strip()))
            outputs.append(list(l[1].strip()))
            tags.append(re.split('\W+', l[2].strip()))
    return inputs, outputs, tags

def find_good_range(a,b):
	mask = [(a[i]==b[i] and a[i] != u" ") for i in range(len(a))]
	if sum(mask) == 0:
		# Some times the alignment is off-by-one
		b = [' '] + b
		mask = [(a[i]==b[i] and a[i] != u" ") for i in range(len(a))]
	ranges = []
	prev = False
	for i,k in enumerate(mask):
		if k and prev:
			prev = True
		elif k and not prev:
			start = i
			prev = True
		elif prev and not k:
			end = i
			ranges.append((start, end))
			prev = False
		elif not prev and not k:
			prev = False
	if prev:
		ranges.append((start,i+1))
	ranges = [c for c in ranges if c[1]-c[0]>2]
	return ranges
def generate_stem():
	return "___"

def get_chars(l):
    flat_list = [char for word in l for char in word]
    return list(set(flat_list))
def best_range(ranges):
    longest_length = 0
    longest_index = 0
    for i,r in enumerate(ranges):
        length = r[1] - r[0]
        if length > longest_length:
            longest_length = length
            longest_index = i
    return ranges[i]

def augment(input_path):
    inputs,outputs,tags = np.array(read_data(input_path), dtype=object)
    temp = [(''.join(inputs[i]), ''.join(outputs[i])) for i in range(len(outputs))]
    aligned = align.Aligner(temp).alignedpairs
    vocab = list(get_chars(inputs + outputs))
    try:
        vocab.remove(u" ")
    except:
        pass

    new_inputs = []
    new_outputs = []
    new_tags = []
    for k,item in enumerate(aligned):
        i,o = item[0],item[1]
        good_range = find_good_range(i, o)
        if good_range:
            new_i, new_o = list(i), list(o)
            r = best_range(good_range)
            s = r[0]
            e = r[1]
            if (e-s>5): #arbitrary value
                s += 1
                e -= 1
            new_stem = generate_stem()
            new_i[s:e] = new_stem
            new_o[s:e] = new_stem
            new_i1 = [c for l,c in enumerate(new_i) if (c.strip() or (new_o[l]==' ' and new_i[l] == ' '))]
            new_o1 = [c for l,c in enumerate(new_o) if (c.strip() or (new_i[l]==' ' and new_o[l] == ' '))]
            new_inputs.append(new_i1)
            new_outputs.append(new_o1)
            new_tags.append(tags[k])
        else:
            new_inputs.append([])
            new_outputs.append([])
            new_tags.append([])
    return new_inputs, new_outputs, new_tags

def find_stems(input_path):
    inputs,outputs,tags = np.array(read_data(input_path), dtype=object)
    temp = [(''.join(inputs[i]), ''.join(outputs[i])) for i in range(len(outputs))]

    with io.capture_output() as captured:
        aligned = align.Aligner(temp).alignedpairs

    vocab = list(get_chars(inputs + outputs))
    try:
        vocab.remove(u" ")
    except:
        pass

    stems = []
    for k,item in enumerate(aligned):
        i,o = item[0],item[1]
        good_range = find_good_range(i, o)
        if good_range:
            r = best_range(good_range)
            s = r[0]
            e = r[1]
            if (e-s>5): #arbitrary value
                s += 1
                e -= 1
            stem = o[s:e]
            stems.append(stem)
    return stems

def get_vocab(strings):
    return sorted(list(get_chars(strings)))

def enumerate_sequence_characters(sequences, vocab):
    lut = {"0":0}
    count = 1
    for character in vocab:
        if character != "0":
            lut[character] = count
            count += 1
    new_sequences = []
    for sequence in sequences:
        new_sequences.append([lut[char] for char in sequence])
    return np.array(new_sequences)

def one_hot_encode_sequence(sequences, vocab):
    length = len(vocab)
    lut = {"0":0}
    count = 1
    for character in vocab:
        if character != "0":
            lut[character] = count
            count += 1
    new_sequences = []
    for sequence in sequences:
        new_sequences.append([[0] * lut[char] + [1] + [0] * ((length - lut[char]) - 1) for char in sequence])
    return np.array(new_sequences), {v: k for k, v in lut.items()}

def get_stem_data(language, set_type="train", hilo=None, data_dir="sigmorphon_data", pad=True):
    if hilo is None:
        if f'{data_dir}/{language}-{set_type}-high' in glob.glob(f'{data_dir}/{language}-{set_type}-*'):
            hilo = "high"
        else:
            hilo = "low"
    dpath = f'{data_dir}/{language}-{set_type}-{hilo}'
    padded_stems = sequence.pad_sequences(find_stems(dpath), dtype=str, maxlen=max_stem_length, padding="post", truncating="post")
    if pad:
        return padded_stems
    else:
        return np.array([np.array(stem) for stem in find_stems(dpath)])

def clean_stems(affirmative_stems, negative_stems):
    excluded_chars = [",","'","/","*","-","1","2","3","4","5","6","7","8","9"]
    affirmative_vocab = get_vocab(affirmative_stems)
    to_be_excluded = []
    for i,stem in enumerate(negative_stems):
        for char in stem:
            if char not in affirmative_vocab or char in excluded_chars:
                to_be_excluded.append(i)
                break
    new_negative_stems = []
    new_affirmative_stems = []
    for i,stem in enumerate(negative_stems):
        if i not in to_be_excluded:
            new_negative_stems.append([character.lower() for character in stem])
    for i,stem in enumerate(affirmative_stems):
        reject = False
        for char in stem:
            if char in excluded_chars:
                reject = True
        if not reject:
            new_affirmative_stems.append([character.lower() for character in stem])
    return new_affirmative_stems, new_negative_stems

def create_stem_dataset(reference_language, other_languages):
    reference_stems = get_stem_data(reference_language)
    other_stems = np.concatenate([get_stem_data(language) for language in other_languages])
    cleaned = clean_stems(reference_stems, other_stems)
    reference_stems = cleaned[0]
    other_stems = cleaned[1]
    combined_stems = np.concatenate([reference_stems, other_stems])
    combined_vocab = get_vocab(combined_stems)
    print(combined_vocab)
    X,lut = one_hot_encode_sequence(combined_stems, combined_vocab)
    labels = np.array([1] * len(reference_stems) + [0] * len(other_stems))
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33)
    return X_train, X_test, y_train, y_test, combined_vocab

def create_single_language_stem_dataset(language):
    reference_stems = get_stem_data(language)
    cleaned = clean_stems(reference_stems, np.array([]))
    reference_stems = cleaned[0]
    combined_vocab = get_vocab(reference_stems)
    print(combined_vocab)
    X,lut = one_hot_encode_sequence(reference_stems, combined_vocab)
    labels = np.array([1] * len(reference_stems))
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33)
    return X_train, X_test, y_train, y_test, combined_vocab, lut


In [3]:
X_train, X_test, y_train, y_test, X_vocab, lut = create_single_language_stem_dataset("english")

['0', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'é', 'ë', 'ö', 'œ']


In [115]:
vocab_len = len(X_vocab)
generator = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dense(vocab_len, activation="softmax")
])
discriminator = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True),
    # keras.layers.Dropout(0.2),
    # keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dense(1, activation="sigmoid")
])
gan = keras.models.Sequential([generator, discriminator])

In [116]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [117]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [118]:
def distribution_to_sequence(batch_output):
    return tf.one_hot(tf.argmax(batch_output, axis=-1), depth = vocab_len)

def train_gan(gan, dataset, batch_size, vocab_len, n_epochs=50):
    generator, discriminator = gan.layers
    discriminator_history = {"loss":[],"accuracy":[]}
    generator_history = {"generated_sequences":[]}
    gan_history = {"loss":[],"accuracy":[]}
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        discriminator_metrics = []
        gan_metrics = []
        generated_images = None
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size,max_stem_length,vocab_len])
            generated_images = distribution_to_sequence(generator(noise))
            X_fake_and_real = tf.concat([generated_images, tf.cast(X_batch, dtype="float32")], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator_metrics = discriminator.train_on_batch(tf.cast(X_fake_and_real, dtype="float32"), y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size,max_stem_length,vocab_len])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan_metrics = gan.train_on_batch(noise, y2)
        for i,v in enumerate(discriminator_metrics):
            discriminator_history[discriminator.metrics_names[i]].append(v)
        for i,v in enumerate(gan_metrics):
            gan_history[gan.metrics_names[i]].append(v)
        generator_history["generated_sequences"].append(generated_images[0])
        history_df = pd.DataFrame({
            "discriminator_loss":discriminator_history["loss"],
            "discriminator_accuracy":discriminator_history["accuracy"],
            "gan_loss":gan_history["loss"],
            "gan_accuracy":gan_history["accuracy"],
            "generated_sequences":["".join([lut[np.argmax(one_hot_char)] for one_hot_char in one_hot_sequence]) for one_hot_sequence in generator_history["generated_sequences"]],
        })
        history_df.to_csv("gan_history.csv")
    return {"discriminator": discriminator_history, "generator": generator_history, "gan": gan_history}

history = train_gan(gan, dataset, batch_size, vocab_len, n_epochs=2000)

Epoch 1/2000


2022-05-04 19:13:05.208722: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 19:13:05.373263: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 19:13:05.495366: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 19:13:07.513595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 19:13:07.829020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 19:13:07.891805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-04 19:13:07.956265: I tensorflow/core/grappler/optimizers/cust

Epoch 2/2000
Epoch 3/2000
Epoch 4/2000
Epoch 5/2000
Epoch 6/2000
Epoch 7/2000
Epoch 8/2000
Epoch 9/2000
Epoch 10/2000
Epoch 11/2000
Epoch 12/2000
Epoch 13/2000
Epoch 14/2000
Epoch 15/2000
Epoch 16/2000
Epoch 17/2000
Epoch 18/2000
Epoch 19/2000
Epoch 20/2000
Epoch 21/2000
Epoch 22/2000
Epoch 23/2000
Epoch 24/2000
Epoch 25/2000
Epoch 26/2000
Epoch 27/2000
Epoch 28/2000
Epoch 29/2000
Epoch 30/2000
Epoch 31/2000
Epoch 32/2000
Epoch 33/2000
Epoch 34/2000
Epoch 35/2000
Epoch 36/2000
Epoch 37/2000
Epoch 38/2000
Epoch 39/2000
Epoch 40/2000
Epoch 41/2000
Epoch 42/2000
Epoch 43/2000
Epoch 44/2000
Epoch 45/2000
Epoch 46/2000
Epoch 47/2000
Epoch 48/2000
Epoch 49/2000
Epoch 50/2000
Epoch 51/2000
Epoch 52/2000
Epoch 53/2000
Epoch 54/2000
Epoch 55/2000
Epoch 56/2000
Epoch 57/2000
Epoch 58/2000
Epoch 59/2000
Epoch 60/2000
Epoch 61/2000
Epoch 62/2000
Epoch 63/2000
Epoch 64/2000
Epoch 65/2000
Epoch 66/2000
Epoch 67/2000
Epoch 68/2000
Epoch 69/2000
Epoch 70/2000
Epoch 71/2000
Epoch 72/2000
Epoch 73/2000


In [53]:
history["gan"]

{'loss': [], 'accuracy': []}

In [47]:
# for seq in generator(tf.random.normal(shape=[batch_size, max_stem_length, vocab_len])):
#     chars = []
#     for onehot in seq:
#         chars.append(lut[np.argmax(onehot)])
#     print(chars)

["".join([lut[np.argmax(one_hot_char)] for one_hot_char in one_hot_sequence]) for one_hot_sequence in generator(tf.random.normal(shape=[batch_size, max_stem_length, vocab_len]))]

['jjjnnnnnnnnnnnnnnnnn',
 'jjjjnnnnnnnnnnnnnnnn',
 'jjnnnnnnnnnnnnnnnnnn',
 'jjnnnnnnnnnnnnnnnnnn',
 'jnnnnnnnnnnnnnnnnnnn',
 'jjnnnnnnnnnnnnnnnnnn',
 'yjjjjjjnnnnnnnnnnnnn',
 'nnnnnnnnnnnnnnnnnnnn',
 'jnnnnnnnnnnnnnnnnnnn',
 'jnnnnnnnnnnnnnnnnnnn',
 'jjjjnnnnnnnnnnnnnnnn',
 'jjjjnnnnnnnnnnnnnnnn',
 'jjjnnnnnnnnnnnnnnnnn',
 'jjjnnnnnnnnnnnnnnnnn',
 'jjjnnnnnnnnnnnnnnnnn',
 'nnnnnnnnnnnnnnnnnnnn']

In [41]:
output = """[[1.64495632e-02 2.29871422e-02 2.80090049e-02 2.09917258e-02
  1.20907919e-02 1.37371328e-02 2.54160389e-02 2.37010177e-02
  3.64153348e-02 7.20336959e-02 1.33804530e-02 5.89604862e-03
  9.87022277e-03 2.61192825e-02 1.95886001e-01 3.99547741e-02
  7.46959671e-02 3.68704125e-02 4.13501523e-02 3.69648561e-02
  3.73360254e-02 1.11349039e-02 5.34372730e-03 2.24610940e-02
  4.99448292e-02 5.15420688e-03 6.46464387e-03 6.66155992e-03
  1.14327772e-02 3.88289168e-02 1.31024662e-02 3.93152274e-02]
 [5.77172905e-04 2.15572873e-04 1.17869501e-03 5.57200867e-04
  8.25689567e-05 8.16560932e-05 2.62634479e-04 1.94973662e-04
  3.84418265e-04 7.58937746e-03 8.31412628e-09 6.01016200e-06
  1.15484494e-04 5.30527381e-04 9.53463018e-01 3.59134376e-03
  1.36423456e-02 1.65962707e-03 2.75825709e-03 1.51432620e-03
  1.74401025e-03 5.05943535e-05 1.74418346e-05 2.15669652e-03
  4.60547768e-03 1.51074104e-07 1.38012174e-05 3.26904992e-05
  8.88111608e-05 6.77606149e-04 2.21232971e-04 1.98625075e-03]
 [1.75035548e-05 1.49619791e-06 2.03041218e-05 5.17065973e-06
  2.29175001e-07 5.74090734e-07 1.96341898e-06 1.76002027e-06
  2.78390189e-06 3.94417933e-04 8.92387206e-12 9.54939772e-09
  7.41089025e-07 4.26056795e-06 9.99074996e-01 1.33551817e-04
  2.18146422e-04 1.08650584e-05 2.71602730e-05 7.29329031e-06
  9.89632372e-06 5.12212104e-08 1.15162191e-08 1.55590569e-05
  3.14483805e-05 1.90502284e-11 1.18745893e-08 3.25887655e-08
  1.39128318e-07 2.74574836e-06 2.46815148e-07 1.65916299e-05]
 [1.79708570e-06 1.17004070e-07 4.92466597e-06 4.51570600e-07
  1.13655059e-08 3.63422714e-08 1.42189251e-07 1.00095228e-07
  2.36944757e-07 8.82790264e-05 2.88515515e-14 2.34767039e-10
  5.59537163e-08 2.47025639e-07 9.99830544e-01 2.56300391e-05
  3.14747449e-05 9.32299031e-07 5.06642164e-06 4.93157927e-07
  1.02966510e-06 1.34348987e-09 2.99691300e-10 1.98299676e-06
  4.51981487e-06 1.24087833e-13 2.58464944e-10 9.46211665e-10
  5.49282486e-09 1.53429980e-07 1.41883376e-08 1.87317914e-06]
 [2.04537400e-06 8.07736384e-08 5.05427488e-06 4.38902504e-07
  9.54301438e-09 3.08233652e-08 1.18130707e-07 8.77337456e-08
  2.03808057e-07 8.38074993e-05 5.85376000e-14 1.81054907e-10
  5.24445127e-08 2.03119427e-07 9.99854922e-01 2.39792444e-05
  1.83998218e-05 7.42159045e-07 2.69541692e-06 2.85167800e-07
  6.37494054e-07 8.43952019e-10 2.06036133e-10 1.71186025e-06
  3.03855381e-06 1.13572088e-13 2.39345599e-10 6.93456403e-10
  4.33373559e-09 9.28113408e-08 9.82852644e-09 1.28119109e-06]
 [1.13281556e-06 5.88976050e-08 4.67861128e-06 3.05986248e-07
  6.63829480e-09 2.11976019e-08 9.24511454e-08 5.81357433e-08
  1.27107469e-07 6.41978550e-05 1.88383736e-14 1.09515473e-10
  4.16551593e-08 1.38256809e-07 9.99886394e-01 1.89285529e-05
  1.51249651e-05 5.12338090e-07 2.37999711e-06 2.22475578e-07
  4.16107923e-07 4.85928908e-10 1.35898751e-10 1.54467011e-06
  2.49984191e-06 6.53325931e-14 1.37094835e-10 5.15979592e-10
  3.14231019e-09 6.37445865e-08 7.55243246e-09 1.06258778e-06]
 [1.00813259e-06 5.08999030e-08 3.61804950e-06 2.75245895e-07
  6.45162279e-09 1.76453412e-08 8.19619927e-08 6.07144628e-08
  1.01701474e-07 5.30186080e-05 3.29668823e-15 1.08237461e-10
  3.00124050e-08 1.17503916e-07 9.99893308e-01 1.56949736e-05
  2.20346228e-05 6.06857384e-07 2.77950085e-06 3.16747844e-07
  4.56022491e-07 6.40416109e-10 1.92686284e-10 1.85594945e-06
  2.95440395e-06 4.13914034e-14 1.72322795e-10 5.03124264e-10
  3.46767193e-09 7.05748349e-08 8.04409428e-09 1.40643920e-06]
 [1.33095159e-06 7.09891452e-08 4.98255167e-06 2.88851624e-07
  6.25731600e-09 2.72622209e-08 9.63928741e-08 7.09098842e-08
  1.49344842e-07 7.78796893e-05 4.25078877e-14 1.14685976e-10
  4.22089457e-08 1.72385512e-07 9.99872684e-01 1.80747047e-05
  1.49642274e-05 5.01270051e-07 2.65352310e-06 2.17948454e-07
  5.23756682e-07 4.97383190e-10 1.35073661e-10 1.47660910e-06
  2.51946949e-06 7.27643586e-14 1.39427872e-10 4.70009143e-10
  2.54414223e-09 7.09271362e-08 8.15325496e-09 1.15793875e-06]
 [8.82921029e-07 4.89557799e-08 3.42936801e-06 2.13351910e-07
  5.43691137e-09 1.84910061e-08 7.37786650e-08 4.73382329e-08
  1.22046572e-07 5.95797646e-05 1.20686610e-14 8.23157514e-11
  2.70418923e-08 1.13173748e-07 9.99892712e-01 1.59482061e-05
  1.76784615e-05 5.02904413e-07 2.76195306e-06 2.03067771e-07
  5.40688347e-07 4.68766248e-10 1.02952522e-10 1.22511995e-06
  2.68823374e-06 4.83096297e-14 1.19354041e-10 4.12727935e-10
  2.40360487e-09 6.63666881e-08 6.93989799e-09 1.03024672e-06]
 [8.83448195e-07 4.42119692e-08 3.65653864e-06 2.49170938e-07
  5.17028642e-09 1.67747505e-08 8.17246075e-08 4.91659691e-08
  1.02430150e-07 5.54987018e-05 7.54926330e-15 9.15744841e-11
  2.44560407e-08 1.08101013e-07 9.99899983e-01 1.43396610e-05
  1.66606478e-05 4.92881895e-07 2.27308760e-06 2.28799706e-07
  4.24219593e-07 4.80669893e-10 1.28292196e-10 1.28960141e-06
  2.50149924e-06 4.03893465e-14 1.14761006e-10 4.18609314e-10
  2.61783395e-09 5.78940842e-08 6.93739821e-09 9.44601084e-07]
 [1.04056835e-06 5.33272839e-08 3.81764903e-06 2.60586461e-07
  6.91056634e-09 2.03616022e-08 8.50863060e-08 4.99029476e-08
  1.10313394e-07 6.32925876e-05 4.58542429e-15 1.27740082e-10
  3.35239996e-08 1.25541447e-07 9.99883890e-01 1.65129968e-05
  1.96988676e-05 5.84240695e-07 3.08512426e-06 2.76360879e-07
  4.91270953e-07 7.35429329e-10 1.90136212e-10 1.71269778e-06
  3.35883237e-06 5.02545901e-14 1.49701571e-10 5.87408511e-10
  3.51691143e-09 7.71416779e-08 8.56890292e-09 1.24577707e-06]
 [1.08995903e-06 5.96728569e-08 3.33391563e-06 2.58823746e-07
  8.81970763e-09 2.51106833e-08 8.54816236e-08 5.51657422e-08
  1.23416257e-07 6.49671492e-05 5.13891882e-15 1.53691573e-10
  3.37676269e-08 1.27419767e-07 9.99878645e-01 1.80090847e-05
  2.22718536e-05 6.77979074e-07 2.92678192e-06 2.92718425e-07
  4.95396932e-07 9.33291222e-10 2.09946768e-10 1.64212020e-06
  3.22701771e-06 6.24029501e-14 1.69421088e-10 6.84218460e-10
  3.22818727e-09 9.29452995e-08 9.48699785e-09 1.35338985e-06]
 [1.71010379e-06 8.27733118e-08 4.70772738e-06 3.68110818e-07
  9.90295757e-09 3.01213774e-08 9.98170506e-08 8.04797011e-08
  1.70242060e-07 7.32831468e-05 3.82703241e-14 1.60437844e-10
  5.23433457e-08 1.80027442e-07 9.99867439e-01 2.27280452e-05
  1.89802158e-05 7.01630199e-07 2.68475287e-06 2.74214869e-07
  5.16799275e-07 7.38755834e-10 1.93469560e-10 1.67705218e-06
  2.91329525e-06 1.07169455e-13 2.01347744e-10 7.73930420e-10
  3.85474186e-09 9.39055624e-08 1.05226885e-08 1.21477001e-06]
 [1.26967711e-06 6.42032418e-08 4.60145293e-06 3.27338483e-07
  5.97468075e-09 2.13371418e-08 9.66232037e-08 6.13632452e-08
  1.24393324e-07 6.54394826e-05 8.61238960e-15 1.28380362e-10
  3.40289361e-08 1.35418588e-07 9.99878287e-01 1.73862663e-05
  2.15830642e-05 5.95593406e-07 3.04751984e-06 3.13408208e-07
  5.59173429e-07 6.40052678e-10 1.75913423e-10 1.58990201e-06
  3.15196212e-06 4.92720861e-14 1.56199220e-10 5.21500232e-10
  3.87990839e-09 7.91586672e-08 8.19429946e-09 1.22753750e-06]
 [1.39909548e-06 8.43695460e-08 3.31858678e-06 3.10755411e-07
  8.77709017e-09 2.59486121e-08 7.88170240e-08 6.24746050e-08
  1.69282529e-07 6.72174792e-05 1.07230365e-14 1.39108350e-10
  4.27720863e-08 1.39972812e-07 9.99873638e-01 2.01629191e-05
  2.22145009e-05 6.18747890e-07 3.62432161e-06 2.67689416e-07
  5.83692724e-07 7.71315123e-10 1.54042723e-10 1.44973637e-06
  3.15933175e-06 5.36455950e-14 1.47122675e-10 6.46856901e-10
  3.03155701e-09 9.97025325e-08 1.02959801e-08 1.32614548e-06]
 [1.76701099e-06 7.74135600e-08 4.72203737e-06 3.72503024e-07
  8.64059668e-09 2.91372562e-08 1.16641971e-07 7.39868682e-08
  1.73180609e-07 8.25538082e-05 2.32031818e-14 2.06995573e-10
  4.05723668e-08 1.57066125e-07 9.99852419e-01 2.19045614e-05
  2.32916573e-05 7.24936228e-07 3.91766616e-06 3.59347581e-07
  6.30350257e-07 9.58581992e-10 2.40853421e-10 1.78178527e-06
  3.67638359e-06 1.04736234e-13 2.35504921e-10 6.36042885e-10
  4.96043961e-09 1.00079298e-07 9.51331280e-09 1.15734292e-06]
 [1.16998456e-06 5.78085917e-08 3.39826533e-06 2.66691529e-07
  6.17373708e-09 1.84271087e-08 9.24771228e-08 6.65153834e-08
  1.20057919e-07 5.03494230e-05 5.80627533e-15 8.95014340e-11
  2.89899056e-08 1.11266388e-07 9.99894977e-01 1.58399089e-05
  2.28791723e-05 5.75147851e-07 2.94708275e-06 2.72107940e-07
  4.75833019e-07 5.81606208e-10 1.34814049e-10 1.65095889e-06
  3.25226779e-06 4.71277883e-14 1.55648536e-10 5.39071565e-10
  3.10174730e-09 7.22601641e-08 7.65961872e-09 1.39620386e-06]
 [1.32586479e-06 6.43987050e-08 4.13422731e-06 2.89572171e-07
  6.31623243e-09 2.11853362e-08 9.48413899e-08 6.33432009e-08
  1.29250978e-07 5.68415635e-05 9.01371805e-15 1.03780581e-10
  3.76532405e-08 1.27435271e-07 9.99886632e-01 1.63955283e-05
  2.32434359e-05 5.66691824e-07 3.12203633e-06 2.88163335e-07
  4.98644283e-07 5.55685664e-10 1.54339500e-10 1.65113931e-06
  3.02286708e-06 5.36067332e-14 1.73227557e-10 5.48566581e-10
  3.71186992e-09 7.72026922e-08 8.68819061e-09 1.34199752e-06]
 [1.13395731e-06 6.09730648e-08 4.22414541e-06 3.05141953e-07
  5.88935656e-09 2.08873345e-08 8.80230218e-08 6.80197516e-08
  1.21363257e-07 6.45133405e-05 1.13278908e-14 1.16963911e-10
  3.11294777e-08 1.26278536e-07 9.99881148e-01 1.73810167e-05
  2.02270949e-05 5.89736544e-07 3.12911425e-06 2.84647200e-07
  4.80155848e-07 5.30417377e-10 1.48744739e-10 1.62440097e-06
  3.00186639e-06 5.58545655e-14 1.63238617e-10 4.55481347e-10
  3.04837933e-09 6.76809435e-08 7.45999440e-09 1.20575601e-06]
 [8.88032616e-07 4.31462936e-08 3.21013772e-06 2.75791876e-07
  5.95393690e-09 1.94355660e-08 7.75918423e-08 5.29396047e-08
  1.03560168e-07 6.38584170e-05 1.02152123e-14 1.19518326e-10
  2.58691379e-08 1.24474155e-07 9.99885321e-01 1.80841125e-05
  1.89554521e-05 5.47481079e-07 2.34981417e-06 2.64198746e-07
  4.69883332e-07 6.12042250e-10 1.74590287e-10 1.38129485e-06
  2.95451719e-06 5.21265533e-14 1.39220732e-10 4.46708975e-10
  2.93516011e-09 6.19175822e-08 6.89370383e-09 8.54079758e-07]]"""
output.replace("  ",", ")

'[[1.64495632e-02 2.29871422e-02 2.80090049e-02 2.09917258e-02\n, 1.20907919e-02 1.37371328e-02 2.54160389e-02 2.37010177e-02\n, 3.64153348e-02 7.20336959e-02 1.33804530e-02 5.89604862e-03\n, 9.87022277e-03 2.61192825e-02 1.95886001e-01 3.99547741e-02\n, 7.46959671e-02 3.68704125e-02 4.13501523e-02 3.69648561e-02\n, 3.73360254e-02 1.11349039e-02 5.34372730e-03 2.24610940e-02\n, 4.99448292e-02 5.15420688e-03 6.46464387e-03 6.66155992e-03\n, 1.14327772e-02 3.88289168e-02 1.31024662e-02 3.93152274e-02]\n [5.77172905e-04 2.15572873e-04 1.17869501e-03 5.57200867e-04\n, 8.25689567e-05 8.16560932e-05 2.62634479e-04 1.94973662e-04\n, 3.84418265e-04 7.58937746e-03 8.31412628e-09 6.01016200e-06\n, 1.15484494e-04 5.30527381e-04 9.53463018e-01 3.59134376e-03\n, 1.36423456e-02 1.65962707e-03 2.75825709e-03 1.51432620e-03\n, 1.74401025e-03 5.05943535e-05 1.74418346e-05 2.15669652e-03\n, 4.60547768e-03 1.51074104e-07 1.38012174e-05 3.26904992e-05\n, 8.88111608e-05 6.77606149e-04 2.21232971e-04 1.9862